In [1]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import json
import glob

## human

In [3]:
# Load your original tweet dataset
column_names = ['target','ids','date','flag','user','text']
human_df = pd.read_csv("/content/drive/MyDrive/fp/training.1600000.processed.noemoticon.csv", encoding='latin-1', names=column_names)

# Reproduce the original shuffling (random_state=42)
human_df_sampled = human_df.sample(frac=1, random_state=42).reset_index(drop=True)

human_df_final = human_df_sampled.iloc[:90000].copy()

human_df_final

,target,ids,date,flag,user,text
0,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
1,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
2,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
3,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
4,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem
...,...,...,...,...,...,...
89995,4,2184347035,Mon Jun 15 15:46:36 PDT 2009,NO_QUERY,PhatGripDotCom,@SGuild Note our website is stuck (server issu...
89996,0,2179132851,Mon Jun 15 08:41:22 PDT 2009,NO_QUERY,slickbacksun,Damn lakers won
89997,4,1835206844,Mon May 18 05:25:32 PDT 2009,NO_QUERY,nikipaniki,@yukai_chou I did have fun at the LG Service C...
89998,4,2062776290,Sun Jun 07 00:15:40 PDT 2009,NO_QUERY,CassieBaby10,Call me the tin man looken for the wizard who ...


In [4]:
human_df_final = human_df_final[['text']]
human_df_final.head()

,text
0,@chrishasboobs AHHH I HOPE YOUR OK!!!
1,"@misstoriblack cool , i have no tweet apps fo..."
2,@TiannaChaos i know just family drama. its la...
3,School email won't open and I have geography ...
4,upper airways problem


In [5]:
human_df_final['tweet'] = human_df_final['text']
human_df_final['AI_Gen_Flag'] = '0'
human_df_final['source'] = 'human'
human_df_final = human_df_final[['tweet','AI_Gen_Flag', 'source']]
human_df_final.head()

<ipython-input-5-fda3150b7831>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  human_df_final['tweet'] = human_df_final['text']
<ipython-input-5-fda3150b7831>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  human_df_final['AI_Gen_Flag'] = '0'
<ipython-input-5-fda3150b7831>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

,tweet,AI_Gen_Flag,source
0,@chrishasboobs AHHH I HOPE YOUR OK!!!,0,human
1,"@misstoriblack cool , i have no tweet apps fo...",0,human
2,@TiannaChaos i know just family drama. its la...,0,human
3,School email won't open and I have geography ...,0,human
4,upper airways problem,0,human


In [ ]:
len(human_df_final)

90000

## 4omini

In [6]:
gpt_files = glob.glob("/content/drive/MyDrive/fp/gpt_batch/4omini_outputs/gpt_batch_part_*_output.jsonl")

In [7]:
gpt_tweets = []

for file in gpt_files:
  with open(file, "r") as f:
    for line in f:
      record = json.loads(line)
      try:
        if record.get("response") and record["response"].get("status_code") == 200:
          content = record["response"]["body"]["choices"][0]["message"]["content"].strip()
          gpt_tweets.append(content)
      except Exception as e:
        print(f"error parsing line in {file}: {e}")


In [9]:
df_gpt = pd.DataFrame({
    "tweet": gpt_tweets,
    "AI_Gen_Flag": 1
})
df_gpt['source'] = 'gpt_4o_mini'

df_gpt.head()

,tweet,AI_Gen_Flag,source
0,"""I'm enjoying some ice cream.""",1,gpt_4o_mini
1,It seems the latest album from Kylie Minogue w...,1,gpt_4o_mini
2,"""@xEvie I only have brown!""",1,gpt_4o_mini
3,"""@MIHappenC Thanks—got the icing going. Typing...",1,gpt_4o_mini
4,"""I get to hang out with my best friend tonight...",1,gpt_4o_mini


In [10]:
len(df_gpt)

28630

## llama

In [14]:
df_llama = pd.read_csv("/content/drive/MyDrive/fp/paraphrased_llama/paraphrased_llama_chunk1.csv")  # update path if needed


In [15]:
df_llama['tweet'] = df_llama[['paraphrased']]
df_llama['AI_Gen_Flag'] = 1
df_llama['source'] = 'llama'
df_llama = df_llama[['tweet','AI_Gen_Flag','source']]
df_llama.head()

,tweet,AI_Gen_Flag,source
0,I'm feeling really unenthusiastic right now. I...,1,llama
1,surely there's someone out there who needs a c...,1,llama
2,Just when you thought it couldn't get any bett...,1,llama
3,I'm really going to miss Liz's incredible qual...,1,llama
4,My new kitten is being quite stubborn when it ...,1,llama


## deepseek

In [16]:
deepseek_path = "/content/drive/MyDrive/fp/deepseek_outputs/chunk3_deepseek_paraphrased.jsonl"

In [17]:
ds_paraphrased= []

with open(deepseek_path, "r") as f:
  for line in f:
    record = json.loads(line)
    if "paraphrased" in record:
      ds_paraphrased.append(record["paraphrased"])

In [18]:
df_deepseek = pd.DataFrame({
    "tweet": ds_paraphrased,
    "AI_Gen_Flag": 1,
    "source": 'deepseek'
})

df_deepseek.head()

,tweet,AI_Gen_Flag,source
0,"""@JTVstacia @moldor @jasonhockey It's great ho...",1,deepseek
1,"""@makensyyy I wanted to share some with you, b...",1,deepseek
2,"""I really want to hit up the rave.""",1,deepseek
3,"""I have a feeling JLS will have a signing on t...",1,deepseek
4,"""Pounding headache hitting hard.""",1,deepseek


In [19]:
len(df_deepseek)

22517

## Combine

In [20]:
df_final= pd.concat([human_df_final, df_gpt, df_llama, df_deepseek],ignore_index= True)
df_final

,tweet,AI_Gen_Flag,source
0,@chrishasboobs AHHH I HOPE YOUR OK!!!,0,human
1,"@misstoriblack cool , i have no tweet apps fo...",0,human
2,@TiannaChaos i know just family drama. its la...,0,human
3,School email won't open and I have geography ...,0,human
4,upper airways problem,0,human
...,...,...,...
171142,"""@loweryc Seriously? She was the one who began...",1,deepseek
171143,"""Feeling super cheerful right now! Haha"" \n\n...",1,deepseek
171144,Here’s a rephrased version of your tweet while...,1,deepseek
171145,"""Almost a whole week without internet!? No way...",1,deepseek


In [21]:
df_final.to_csv('tweet_data_final_with_source.csv')

In [22]:
!ls

drive  sample_data  tweet_data_final_with_source.csv
